<a href="https://colab.research.google.com/github/nlp2bok/fintech_/blob/lmh_dev/%EC%9D%98%EC%82%AC%EB%A1%9D%EC%84%B8%EC%85%98%EB%B6%84%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import os
import pandas as pd

In [0]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return ' '.join(sentences), text

In [0]:
def mydoc(file_path):
    minutes = open(file_path, encoding=u'utf-8').read()

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여,?|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,
                    re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    # pos = re.search('((\((다|라)\) )?.?통화정책방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에 관해 다음과 같은 토론이 있었음.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책.?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,
                    re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,
                    re.MULTILINE)
    s6 = pos.start() if pos else -1
    # pos = re.search('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    # s7 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section1 = section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section2 = section
    section2, section2_txt = tidy_sentences(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section3 = section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section4 = section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section5 = section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    #section6 = section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)

    print(section1)
    #print(' ==> text processing completed: {}'.format(filename))
    doc = []

    if any(section_texts):
        for s, (section, sentences) in enumerate(zip(sections, section_texts)):
            for p, text in enumerate(sentences):
                #row = (filename, mdate, rdate, section, p, text)
                row = ('filename','mdate','rdate',section,p,text)
                doc.append(row)
    else:
        print('Empty!')
        
    return section_texts

#데이터 저장

In [0]:
def fileopen(path):
    f = open(path,'r', encoding='utf-8')
    line = f.read()
    f.close()

    return line 

In [0]:
workDIr = os.path.abspath('/Users/minhyeon/Desktop/Data_Preprocessing_MPB/MPB_Contents/')
columns = ['rDate', "contents",'Economic Situation', 'Foreign Currency', 'Financial Markets',
           'Monetary Policy', 'Participants’ Views', 'Government’s View']
df = pd.DataFrame(columns = columns)
#df = pd.DataFrame()
#날짜 원문 토큰 엔그램
for dirpath, dirnames, filenames in os.walk(workDIr):
        for filename in filenames:
            file_path = "MPB_Contents/"+filename
            rDate = filename[:8]
            fType = filename[-4:]
            try:
                if fType == ".txt" : 
                    raw = fileopen(file_path)
                    section_texts = mydoc(file_path)
                    data = [rDate, raw, section_texts[0], section_texts[1], section_texts[2], section_texts[3], section_texts[4], section_texts[5]]
                    df.loc[len(df)] = (data)
                else :
                    continue
                #파일 타입 체크
            except Exception as ex:
                print(filename + '파일 read 에러가발생 했습니다', ex)




일부 위원은 최근 소득여건 및 소비심리의 개선에도 불구하고 높은 가계부채 부담 등으로 인해 소비의 회복이 지체되고 있다고 지적한 후, 지난 2003～04년 중에도 카드사태 등으로 민간소비가 크게 위축된적이 있는데, 당시에는 가처분소득 대비 가계부채 비율이 높지 않아 가계부채에따른 양(陽)의 유동성 효과가 부(負)의 원리금 효과보다 크게 나타나면서 가계부채가 오히려 소비둔화를 완화시키는 역할을 수행하였으며, 이에 힘입어 이듬해성장률이 회복되자 민간소비도 즉각적으로 반등할 수 있었던 반면, 최근에는 동부채비율이 매우 큰 폭으로 상승함에 따라 향후 경제여건이 호전되더라도 민간소비의 회복을 장기간 기대하기 어려운 임계점에 도달하지는 않았는지 우려되는 상황이므로 이를 면밀히 점검해 볼 필요가 있다는 의견을 밝혔음. 이와 관련하여 동 위원은 미국, 영국, 일본 등 주요국들의 경우 가계부채의디레버리징(deleveraging)이 진행되면서 가처분소득 대비 가계부채 비율이 크게 낮아짐에 따라 주택시장과 민간소비의 구조적 반등을 기대할 수 있는 상황에 이르렀다고 지적한 후, 민간소비의 회복 차원에서라도 가계부채에 대한 디레버리징의중요성을 인식할 필요가 있다는 견해를 나타내었음. 아울러, 동 위원은 최근 소비자물가 상승률이 공급측면에서의 인플레이션 압력 완화 등에 힘입어 예상을 하회하는 낮은 수준을 나타내고 있으며, 하반기 중에도 이러한 추세가 이어질 것으로 보임에 따라 7월 경제전망 시 동 전망치는 하향 조정될 가능성이 높다고 지적하였음. 이와 관련하여 동 위원은 최근의 물가안정세는 주로 공급측 요인에 기인하고있다는 점에서 지금이 공공요금을 현실화할 수 있는 좋은 기회일 수 있다고 지적한 후, 일반적으로 공공요금 인상률이 소비자물가 상승률에 수렴하는 모습을 보여 왔으나, 금융위기 이후에는 공공요금 인상률이 소비자물가 상승률을 계속 하회하여 왔다면서, 물가상승률을 감안하여 계획적으로 적절한 일정을 세워 공공요금을 현실화한다면 최근 급락하고 있는 소비자물가 상승률을 평탄화(f


일부 위원은 최근 몇 년간 가계소비성향이 고령화 진전, 가계부채 및 주거비 부담, 저성장 지속 등의 영향으로 하락하여 왔다는 최근 분석결과를 언급하면서, 가계소비성향의 하락은 기본적으로 인구구조 변화 등에 기인한 것이므로 인위적인 소비 진작보다는 안정적 소비기반 확보 차원에서 연금 등 노후복지를 확충하는 한편 취약계층에 대한 소득재분배 기능을 제고할 필요가 있다는 견해를 나타내었음. 다른 일부 위원은 최근 소비심리지수나 경기선행지수를 보면 경제주체들의심리 악화가 두드러지고 있는 것으로 보이는데, 그 배경을 두고 대내외 여러 불확실성 요인이 작용한 결과라는 해석이 있는가 하면, 수출 등 업황 부진 및 이에따른 구조조정의 파장으로 이해하거나, 내수 견인의 중요한 축인 부동산 경기가둔화된 데 따른 영향으로 보는 등 다양한 시각이 있는데, 만일 가계나 기업이 불확실성에 대한 인식을 바꾸지 않고 자기강화적 동태적 적응에 머물러 있게 될 경우 우리 경제에 어려움을 가중시킬 우려가 있다고 지적하면서, 관련부서의 견해를 물었으며,이에 대해 관련부서에서는 최근의 소비 및 투자 심리 위축에는 앞서 언급한다양한 요인들이 복합적으로 영향을 미치고 있는 것으로 보인다고 답변하고, 대내외 불확실성이 증대되면서 여건 변화에 따른 경제심리의 민감도가 높아지고 있다는 점 등을 감안할 때 앞으로 글로벌 여건 호전이나 수출 개선이 이뤄질 경우경제주체들의 심리가 빠르게 회복될 가능성도 있다고 덧붙였음. 이어서 동 위원은 우리나라의 수출부진이 예상보다 길어질 수 있다는 시각도있다면서, 그 근거로 첫째, 세계경제 회복을 위해서는 수요 개선, 생산성 향상, 현재 진행 중인 구조조정의 완료 등이 선결되어야 하나 이러한 문제 해결에는 상당한 기간이 소요될 수 있다는 점, 둘째, 유가의 본격적인 재상승 국면 진입을 위해서는 글로벌 수요 확대, 재고수준 감소, 공급질서 안정화 등이 요구되나 이 역시단기간 내에 해결되기 어렵다는 점, 셋째, 우리나라 수출이 기존제품 공급경쟁 심화, 교역시장 창출의 한계, 글

일부 위원은 최근 건설투자 등 일부 경제지표가 지난해 12월 예상했던 것보다 부진한 것으로 나타난 점, 가계부채의 원리금상환이 민간소비에 부정적인 영향을 미칠 수 있는 점 등이 2012년도 경제성장 전망에 미치는 영향에 대해 검토해볼 필요가 있다는 견해를 밝혔음. 다른 일부 위원은 소매판매 설비투자 등 내수지표의 하락세가 뚜렷하고 수출이둔화되고 있으며 생산지수도 양호한 수준에 있다고 보기 어려운 점 등을 고려하면국내 경기가 본격적인 하강국면에 접어들었을 가능성이 있다고 언급하였음또 다른 일부 위원은 최근 내수 및 수출이 모두 부진한 모습을 보이고 있음에 비추어 볼 때 금년 1/4분기의 전기대비 성장률이 마이너스가 될 가능성이 있는지 물었으며,이에 대해 관련부서에서는, 지난 4/4분기중 소비와 투자의 성장기여도가 전기대비 마이너스를 보인 것은 대외 불확실성에 대한 민감도가 크게 높아졌기 때문이며, 대외 불확실성 자체는 큰 변동이 없는 상황이므로 앞으로 경제주체들의 심리가 더 이상 위축되지 않으면서 당초 전망했던 성장패턴을 보일 것이며, 1/4분기성장률은 전기대비 플러스로 예상된다고 답변하였음한편, 일부 위원은 지난해 4/4분기중 내수 수출 등이 부진한 모습을 보인 것은 이상고온에 따른 겨울철 의류 판매 부진, 한·미 FTA에 따른 세금인하 기대로나타난 자동차 수요의 이연, 환율 상승으로 인한 자본재 수입의 연기, 태국 홍수에 따른 컴퓨터 생산 차질 등의 특이요인이 상당한 영향을 미쳤을 수 있으므로향후 경기상황 점검 및 전망시 이를 충분히 감안하여야 할 것이라는 의견을 나타내었음일부 위원은 2013년부터 새로 시작되는 3년간의 중기 물가안정목표를 금년중 새로 설정해야 하는데 통화정책의 시차 등을 감안할 때 충분한 시간을 가지고 정부와- 3 -협의하여 결정할 필요가 있다고 발언하였으며, 다른 위원들도 이에 동의하였음아울러, 동 위원은 과거 선거가 있던 해에는 물가가 평소보다 높아지는 경향이있었음을 감안할 때 금년도 물가가 당초 전망치를 상회할 가능성이 있으므로 각종

일부 위원은 금년 1/4분기 중 정부의 재정 조기집행에도 불구하고 세입이 더빠르게 늘어 재정수지 흑자규모가 전년동기보다 확대되었으며, 특히 소득세수입증가율이 명목GDP 성장률을 크게 상회한다고 언급하였음. 이러한 세수 증가세가작년에 이어 금년에도 지속되고 있는 주된 이유와 향후에도 지속될 것으로 보는지 관련부서에 질의하였음. 이에 대해 관련부서에서는 금년 1/4분기 중 세수 호조는 경기 개선, 소비 증가 등으로 소득세 및 부가가치세 수입이 상당 폭 증가한 데 기인한 것으로 파악되며, 재정수입을 전망하는 데 상당한 불확실성이 있으나 작년 정부의 세수 전망시와 현재 시점의 예상 경제성장률이 큰 차이가 없다는 점에 비추어 보면, 향후세수는 정부가 당초 예상한 규모보다 크게 늘어나기는 어려울 것으로 보인다고답변하였음. 이어서 동 위원은 이와 같은 세수 증가 추세가 지속된다면 최근 논의되고 있는 추가경정 예산이 편성된다고 하더라도 금년도 재정정책 기조가 확장적이라고보기가 어려울 수도 있으며, 추경 편성이 어느 정도 성장 전망경로의 상방 리스크 요인으로 작용할 수 있을지도 불확실해 보인다고 언급하였음. 또한 최근의 세수 증가 추세를 경기적 요인, 부동산 및 금융 사이클(cycle) 요인, 인터넷 상거래확산에 따른 투명성 제고 등의 구조적 요인으로 나누어 세밀하게 살펴볼 필요가있다는 의견을 나타내었음. 향후 부동산 및 금융 사이클이 긴축적으로 전환될 가능성이 높아 보인다고 언급하면서, 부동산 경기와 신용 순환의 영향을 배제한 금융중립적인 재정수지를 추정하여 이를 바탕으로 재정정책을 평가해 볼 필요도 있다고 당부하였음. 아울러 자금순환 통계상 정부부문의 자금잉여가 증가하고 분배측면에서도 정부의 소득비중이 높아지고 있는 상황이므로, 앞으로 정부지출이 우리 경제의 생산성을 높이는 방향으로 얼마나 효율적으로 사용되는지 여부가 중요한 이슈가 될 것으로 보인다는 의견을 나타내었음. 최근 정부지출 구조를 살펴보면 사회복지 지출의 증가세가 현저한데 이는 고령화 등으로 일부 불가피한 측면

일부 위원은 최근 들어 주요 실물지표들이 등락을 반복하고 있고, 경기둔화에 대한 우려가 여전히 크다는 점에서 경기부양책에 대한 일반의 기대가 높아지고 있으나, 금리정책 등 거시정책수단의 경우 가계부채, 부문별 양극화 등의 제약요인으로 인해 경기진작 효과는 제한적인 가운데부작용이 더 클 수 있다는 점에서 추진하기가 쉽지 않을 것이라고 지적한 후, 이러한 상황에서 부동산시장 활성화 대책이 소비진작 차원에서 추진될 가능성이 있으므로 부동산 경기가 민간소비에 미치는 영향을 점검하는 한편, 바람직한 정책방향 및 각 정책수단별 장단점과 우선순위 등을 면밀히 검토해 볼 필요가 있다는의견을 나타내었음. 이어서, 동 위원은 당행 분석에 따르면 주택가격 상승이 민간소비를 늘리는효과는 미미한 반면, 전월세 가격이 상승할 경우 민간소비가 상대적으로 크게 감소하는 것으로 나타났으며, 특히 전세가격의 경우 단기적인 소비감소 효과가 매우 큰 것으로 분석되었는데, 이러한 차이는 매매가격의 상승이 다수의 가계에게는 실현되지 않은 소득증가라는 점, 전월세 가격이 직접적으로 임차가구의 소비여력을 제약하게 된다는 점, 또한 전월세 가격 상승은 임대차가구 간의 소득이전이기는 하지만 양 가구 간에는 한계소비성향에서 차이가 있고 가격상승 시에는자금운용의 불확실성이 증가한다는 점 등에 기인하는 것으로 보인다고 지적한후, 최근 주택 매매가격이 하락세를 보이고 있는 점, 전월세가격 상승세가 과거보다 크다는 점, 전세시장이 전월세 시장으로 바뀌고 있다는 점 등으로 인해 동 효과가 가속화되었을 가능성이 있다면서, 최근의 주택매매시장의 상황 및 전월세시장의 구조변화가 가계소비 및 민간소비에 미치는 영향 등에 대해 면밀히 분석해 볼 필요가 있다는 견해를 피력하였음. 아울러, 동 위원은 거시경제 안정 측면에서 바람직한 부동산정책이 무엇인지점검해 볼 필요가 있다면서, 그 구체적인 내용을 다음과 같이 언급하였음. 먼저, 우리나라는 가계의 디레버리징(deleveraging)이 일어나지 않았다는 점에서 물가수준을 넘는 주

...

일부 위원은 금년 1/4분기 수출이 높은 증가율을 기록하였는데 단가 상승뿐만아니라 물량 증가에도 상당부분 힘입은 데다 IT 경기의 확장, 세계경제의 회복에기인한 부분도 큰 것으로 나타났다고 언급하였음. 앞으로도 수출이 당분간 호조세를 이어갈 것으로 보인다면서, 이에 따른 영향이 내수로는 어떠한 경로를 통해얼마나 파급될 것으로 예상하는지 관련부서에 물었음. 이에 대해 관련부서에서는 최근의 수출 증가가 수입의존도가 높고 고용창출력이 낮은 반도체 및 석유화학에 의해 주도되고 있어 아직까지 내수나 고용으로의 파급효과는 제한적인 것으로 보인다고 설명하고, 다만 앞으로도 이들 품목의수출 호조세가 지속될 것을 감안하면 기계장비 등 연관 산업이나 종사자의 임금상승 등을 통해 그 효과가 조금씩 확산될 것으로 판단된다고 답변하였음. 이에 동 위원은 금년의 경우 수출이 예상보다 확대되고 그 파급효과도 좀 더확산된다면 수입유발 효과를 차감한 수출의 성장기여도도 지난 4월 전망보다 높아질 것으로 보이는지 관련부서에 물었으며,이에 대해 관련부서에서는 올해 수출의 성장기여도가 당초 전망보다 높아질것으로 예상된다고 답변하였음. 이어서 동 위원은 금년 하반기에 일자리 창출을 위한 추경이 실시된다 하더라도 올해 세수 증가에 따른 긴축효과를 감안하면 성장률에 기여하는 정도가 크지 않을 수도 있다는 의견을 나타내었음. 이에 대해 관련부서에서는 세수의 증가가 세제 개편과 같은 외부충격이 아니라 경제활동의 결과로 나타난 경우에는 성장률에 대한 긴축효과가 미미한 것으로분석되었다고 설명하고, 이러한 점에서 앞으로 추경이 실시될 경우에는 성장률에긍정적인 영향을 미칠 것으로 보인다고 답변하였음. 또한 동 위원은 서울 등 수도권을 중심으로 아파트 매매가격의 오름세가 확대되고 그 거래량도 늘어나고 있다고 언급하였음. 최근 몇 년 전의 가격상승기와는 달리 서울과 지방이나 매매와 전세 간의 가격상승률 격차가 확대되고 있어 경- 3 -기적인 요인에 재건축 및 개발호재와 관련한 투기적인 요인도 일부 가세하고 있는 것


일부 위원은 이번 달 수정전망에서 금년도 성장률 전망치를 7월 전망시보다 큰 폭 하향조정한 이유에 대해 물었으며,이에 대해 관련부서에서는 당초 7월 전망시에는 향후 대외여건이 개선되면서3/4분기 GDP가 전기대비 1% 이상 성장할 것으로 예상하였으나, 실제로는 유로지역 위기상황이 장기화되면서 3/4분기 실적이 크게 낮아진 것으로 추정됨에 따라큰 폭의 하향조정이 불가피하였다고 답변하였음. 이어서, 동 위원은 GDP 구성항목중 특히 설비투자의 하반기 전망치를 큰 폭하향조정한 이유가 무엇인지 물었으며,이에 대해 관련부서에서는 설비투자는 불확실성 요인에 가장 크게 좌우되는데, 하반기 들어서도 불확실성이 계속 이어짐으로 인해 설비투자 실적이 상당히부진한 것으로 나타남에 따라 대폭 하향조정하게 된 것이라고 답변하였음. 또한 동 위원은 금년 들어 당행의 성장률 전망치가 큰 폭의 하향조정을 거듭하게 된 것은 주요 전제치인 세계경제성장률에 지속적인 전망오차가 발생한 데주로 기인한 것으로 파악된다고 지적한 후, 당행이 경제전망시 참고하는 IMF 등주요 국제기구의 세계경제성장률 전망치는 주요 전망대상국의 정부 등과 협의를거친 후 나오기 때문에 위기상황에서는 자국 경제를 보다 긍정적으로 보려는 시각이 반영될 가능성이 크다는 점에 유의하여야 할 것이라는 의견을 밝혔으며,이에 대해 관련부서에서는 IMF 등의 전망치를 그대로 이용하지 않고 미국 유로 중국 등 주요국의 상황을 감안하여 전망의 전제를 설정하고 있다고 설명한후, 다만 글로벌 경기상황을 예측하기가 매우 어렵기 때문에 계속 차이가 발생하고 있다고 답변하였음. 이에 대해 동 위원은 세계경제성장률 전체치의 정도(精度)를 높이기 위해 이부문에 대한 독자적인 전망역량을 강화할 필요가 있다는 견해를 피력하였음. - 3 -아울러, 동 위원은 우리나라의 수출은 1980년대 이후 최근까지 지속적으로 세계수출 증가율을 큰 폭 상회하였고, 세계교역이 위축되었을 때에도 그 위축폭이상대적으로 작았으며, 반등시에는 훨씬 더 큰 폭으로 반등하여 왔었는

일부 위원은 GDP성장률이 지난해 2%에 이어올해도 2.6%에 그칠 것으로 예상됨에 따라 저성장 기조의 장기화에 대한 우려가많다고 지적한 후, 이와 같은 저성장 국면의 장기화가 경기순환적 요인과 구조적요인 중 어느 것에 주로 기인하는 것으로 보는지 물었으며,이에 대해 관련부서에서는 우리나라는 GDP에서 차지하는 수출비중이 매우높기 때문에 대외여건이 개선되지 않으면 국내경기도 따라서 부진할 수밖에 없다고 설명한 후, 지금의 상황도 2008년 리먼사태 이후 선진국의 경기가 부진을 지속하는 등 대외여건이 개선되지 못한 데 주로 기인한 것이라는 점에서 경기순환적 측면이 강한 것으로 생각되며, 아울러 소비부진 등과 관련하여서는 가계부채문제, 고령화 등 구조적인 문제도 일부 가세하고 있는 것으로 보인다고 답변하였음. 또한, 동 위원은 민간소비의 성장기여율이 2001～07년 평균 50%에 근접하였으나 2010～11년 중 30%대로 대폭 하락하였고, 지난해와 올해 다시 회복하는 것처럼 보이고 있으나, 동 기간 중 성장률 자체가 크게 하락하였다는 점에 기인한반사효과에 불과하다고 지적한 후, 지출항목 중 수출이 대외여건 악화에도 불구하고 선방하고 있고, 투자도 정부의 정책의지 등에 힘입어 하반기 이후 확대될가능성이 있다는 점을 감안하면, 잠재성장률 달성을 위해서는 민간소비의 회복이긴요한 과제라고 할 수 있다는 의견을 밝혔으며, 아울러 실제 추산을 해보면 민간소비가 금융위기 이전 수준인 4% 정도로 확대될 경우를 가정하면 성장률이 잠재성장률 수준에 도달하는 것으로 나타났다고 첨언하였음. 이와 관련하여 동 위원은 민간소비의 위축은 성장률 자체를 하락시킬 뿐만아니라 경기변동의 완충기능을 훼손하고, 체감경기 및 경제심리를 위축시킴으로써 경기부진의 악순환을 초래하는 한편, 장기적으로는 투자심리 약화로 자본축적이 저하됨으로써 잠재성장률이 하락할 수도 있는 등 심각한 문제를 야기할 수 있다고 지적한 후, 민간소비를 회복시킬 수 있는 보다 구체적이고 현실성 있는 방안을 마련해 줄 것을 당부하

일부 위원은 10월중 일반인 기대인플레이션이지난달보다 0.1%포인트 낮아지면서 동 자료 조사 이후 최저수준이라고 지적하고,비록 통계적 유의성에 다소간에 의문이 있을 수 있지만 기대인플레이션이 낮아진것이 실제 물가를 자기실현적으로 낮출 가능성이 있다는 점에 유의할 필요가 있다는 견해를 피력하였으며,이에 대해 관련부서에서는 일반인 기대인플레이션이 서베이(survey)를 통해 조사되어 오차가 있을 수 있다는 점, 일반인 기대인플레이션이 최근 물가의 영향을받는다는 점, 공공요금 및 집세에 대해서는 높은 수준의 불안심리가 상존하고 있다는 점 등을 고려해 보면 일반인 기대인플레이션의 추세적 하락 여부는 조금 더지켜봐야 판단할 수 있겠다고 답변하였음. 이어서 동 위원은 10월중 근원인플레이션율도 전달에 비해 소폭 낮아졌다고지적한 후, 경기 부진으로 다양한 형태의 할인판매가 이루어지고 있는데 이것이소비자물가지수(CPI)에 실질적으로 반영되고 있는지 물었으며,이에 대해 관련부서에서는 10월 CPI에 대형 유통업체의 할인판매가 반영된것으로 알고 있다고 답변하였음. 이와 관련하여 실제 물가상승률과 지표 물가상승률간에 괴리가 있다는 외부의 시각을 소개한 후, 당행의 가장 중요한 책무가 물가안정인 만큼 앞서 언급한기대인플레이션 및 근원인플레이션 하락, 실제 물가상승률과 지표 물가상승률간의 괴리 등에 대해 보다 유의할 필요가 있다는 견해를 제시하였음. 다른 일부 위원은 지난 8월과 10월중 당행의 기준금리 인하가 금융시장에 긍정적인 영향을 미치고 있는 것으로 평가되나 실물경기에 미치는 파급시차를 감안하더라도 경제주체의 심리지표가 개선되지 않고 있다는 점에는 우려가 있다는 의견을 나타내었음. 이어서 동 위원은 그간 안정된 모습을 보이던 기대인플레이션이 낮아지고 있고, 물가도 목표범위를 계속 하회하여 다양한 이슈가 제기되고 있다고 언급한 후,공공부문에서 물가인상이 억제되어 온 점 등 수요 및 공급 측면을 고려해 볼 때구조적으로 디플레이션(deflation)이 크게 우려되는 상황은 아니라고


일부 위원은 월세 등 가계주거비 증가율이 빠르게 오르고 있다고 지적하면서, 가계의 주거비 부담 확대는 소비 등 경제에 부정적인 영향을 초래할 가능성이 높으므로 원활한 소비 흐름을 지원한다는 차원에서 주거비의 과도한 상승을 방지할 수 있는 대책을 국외 사례 등을 참고하여 마련할 필요가 있다는 견해를 제시하였음. 이에 대해 관련부서에서는 현재 가계주거비 통계는 월세 지출 기준으로 집계되는 것으로 알고 있다고 설명한 후, 전세에서 월세로 전환되는 가구들이 늘어나면서 가계주거비 증가율이 높은 수준을 기록하고 있으나 전월세 전환율이 하락하고 있다는 점을 고려해 보면 향후 전월세 전환이 안정될 경우 가계주거비 증가율이 낮아질 것으로 보고 있다고 답변하였음. 이와 관련하여 다른 일부 위원은 정부에서 서민의 주거비 부담을 완화하는방안을 마련한 바와 같이 주거비 부담 문제가 우리 경제의 중요한 현안과제인 것은 사실이지만, 현재 가계주거비 통계에는 월세 지출만 포함되어 있다는 점, 주거형태를 월세로 전환할 경우 전세 자금 조달 부담이 경감되거나 이자소득이 발생할 수 있다는 점 등을 균형 있게 고려하지 않고 가계주거비 증가율 수치를 단순하게 해석할 경우 주거비 부담을 과대평가할 소지도 있다는 의견을 덧붙였음. 이어서 동 위원은 수출 금액이 8월 들어 전년동월대비 크게 감소한 가운데최근 물량기준(선박제외)으로 수출 증가율이 낮아지고 있다는 점에 유의하여 향후수출 추이를 물량기준으로도 면밀히 점검할 것을 당부하였음. 또한 동 위원은 중국경제의 경기 둔화와 위안화 절하로 인해서 국내 경제가상당한 어려움에 처할 것이라는 우려가 시장에 과도하게 확산되고 있다고 지적하고, 위안화의 추가 약세 등 중국경제 리스크가 향후 현재화될 경우 우리 경제에미칠 영향을 통계 등 사실에 기반을 두어 객관적으로 분석하고 그 결과를 일반에공개함으로써 경제주체의 과도한 불안심리가 우리 경제의 변동성 확대로 이어질가능성을 차단하는 것이 바람직하며, 다른 한편으로는 예상되는 시나리오에 맞추어 선택 가능한 정책수단

In [0]:
df.to_csv("/Users/minhyeon/Desktop/Data_Preprocessing_MPB/MPB_parsing.csv", header=True, index=False)

In [0]:
text = pd.read_csv("MPB_parsing.csv",encoding="utf-8")

In [0]:
print(section_texts)

('일부 위원은 통화당국의 입장에서 경기변동성과 단기추세에 대해 정확히 모니터링하는 것에 못지않게 잠재성장률의 흐름을 파악하는 것이 중요하다고 언급하면서 우리나라가 금융위기 이후 중요한 구조적 변화를 겪고 있으므로 잠재성장률이 어떻게 변화하고 있는지에 더욱 관심을 기울일 필요가 있다고 당부하였음. 이와 관련하여 중요한 현상들의 근본 원인, 예를 들면 비IT 부문의 저조한 설비투자가 경쟁력 부재에 기인하는 것인지, 아니면 기업들이 세계경제 흐름을 제대로따라가지 못한 데 기인한 것인지 등을 면밀히 파악할 필요가 있다고 강조하였음. 이어서 동 위원은 신흥국 금융불안의 확산은 우리 경제의 직접적 하방 리스크로 작용할 가능성이 높지 않아 보인다고 언급하면서, 이보다는 주요국의 통화·재정정책 변화에 따른 국제금융시장의 변동성 확대에 더 주의할 필요가 있다는의견을 밝혔음. 또한 동 위원은 최근 외부에서도 높은 관심을 보이는 관리물가의 경우 단기적 변동성이 제거될 수 있는 과거 5년 동안 기간을 보면 전반적으로 소비자물가를 하회하는 구조적인 흐름을 나타내고 있다는 점에서 앞으로 관리물가의 중요성이 보다 부각되어야 할 것으로 보인다는 의견을 나타내었음. 아울러 관리물가를반영할 경우 필립스곡선의 유효성이 더 강화된다는 분석결과도 있다고 덧붙였음. 다음으로 동 위원은 선진국의 확장적 재정정책이 신흥국에 미치는 영향에 대한 관련부서의 분석이 상당히 유익했다고 평가하면서, 더 나아가 선진국의 거시경제정책이 확장적 재정정책과 긴축적 통화정책의 조합으로 운영되고 있는데, 이러한 정책조합이 우리 경제에 미치는 영향을 주의 깊게 봐야 할 것이라고 언급하였음. 아울러 G20 회의에서도 선진국의 거시경제정책의 조화가 중요한 이슈로 부각되고 있다고 부연하였음. 한편 동 위원은 내년에 15∼59세 생산가능인구가 상당 규모 감소할 것으로예상되는 점을 감안할 때, 관련부서가 전망한 내년 취업자수 증가규모가 현실화- 3 -되기 위해서는 60세 이상 취업자수 증가, 경제활동참가율 상승 등을 전제하여야한다

In [0]:
text.head(30)

,rDate,contents,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participants’ Views,Government’s View
0,20080110,minute2.hwp금융통화위원회 의사록2008년도 제2차 회의1. 일 자 ...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 감안할 때 이번 달에는 콜금리 목표를 현 ...,NaN
1,20131212,2012년도 제21차 금융통화위원회2013년도 제23차금융통화위원회(정기) 의사록한...,NaN,일부 위원은 최근 SWIFT 자료에 따르면 중국 위안화가 전통적인 무역금융 통화 순...,NaN,NaN,일부 위원은 세계경제의 중요 이슈 중하나인 세계경제의 장기 저성장(secular s...,NaN
2,20120413,제7차 금통위 의사록.hwp- 1 -금융통화위원회 의사록2012년도 제7차 회의1....,NaN,NaN,NaN,NaN,일부 위원은 향후 금융안정보고서 작성시 개선 필요사항에 대하여 금융안정보고서가 여러...,NaN
3,20130613,2012년도 제21차 금융통화위원회2013년도 제11차금융통화위원회(정기) 의사록한...,일부 위원은 최근 소득여건 및 소비심리의 개선에도 불구하고 높은 가계부채 부담 등으...,일부 위원은 향후 미국 양적완화정책의 출구전략과 관련하여 시기에 대해서는 논란이 있...,일부 위원은 글로벌 금융위기 이후 국내 금융산업을 보면 외형적으로는 큰 문제가 없으...,NaN,일부 위원은 최근 소득여건 및 소비심리의 개선에도 불구하고 높은 가계부채 부담 등으...,NaN
4,20120308,제5차 금통위 의사록.hwp- 1 -금융통화위원회 의사록2012년도 제5차 회의1....,NaN,NaN,NaN,NaN,일부 위원은 최근의 선진국 및 개도국경제상황 변화와 관련하여 우리나라의 1/4분기 ...,NaN
5,20160714,의사록2016년도 제14차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 ...,일부 위원은 이번 성장전망에서 추경 등 확장적 거시정책이 상방요인으로 거론되었는데 ...,일부 위원은 브렉시트로 인한 국제금융시장의 불안심리 확산이 예상보다 제한적인 것으로...,일부 위원은 우리나라의 30년물 금리가 미국보다 낮은 것이 향후 우리경제에 대한 비...,NaN,일부 위원은 이번 성장전망에서 추경 등 확장적 거시정책이 상방요인으로 거론되었는데 ...,NaN
6,20160216,2016년도 제3차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,일부 위원은 최근 대외 불확실성이 높아지고국내 실물지표도 경상수지 외에는 뚜렷한 개...,일부 위원은 도이치은행의 최근 상황과 이에 따른 시장의 영향에 대한 관련부서의 견해...,일부 위원은 기업 구조조정이 진행되고 있는 가운데 작년 4/4분기중 은행의 중소기업...,NaN,일부 위원은 최근 대외 불확실성이 높아지고국내 실물지표도 경상수지 외에는 뚜렷한 개...,NaN
7,20091210,ACARWXAQSUDX.hwp- 1 -금융통화위원회 의사록2009년도 제 차 회의2...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려할 때 이번 달에도 한국은행 기준금리를...,NaN
8,20160310,의사록2016년도 제5차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자...,"일부 위원은 최근 몇 년간 가계소비성향이 고령화 진전, 가계부채 및 주거비 부담, ...",일부 위원은 지난 5년간 미 달러화대비 주요국 통화의 흐름을 살펴보면 원화가치의 절...,일부 위원은 통화정책은 금리뿐만 아니라 주가에 대해서도 유의한 영향을 미친다는 것이...,NaN,"일부 위원은 최근 몇 년간 가계소비성향이 고령화 진전, 가계부채 및 주거비 부담, ...",NaN
9,20080508,LNEVGLIFETSD.hwp금융통화위원회 의사록2008년도 제 차 회의10일 자 ...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점들을 종합해 볼 때 경기상승세 둔화가 점차가시화되면서 성...,NaN
